List Nodes in cluster

In [ ]:
%%bash 
kubectl get node

Create and set a namespace to work in

In [ ]:
%%bash
kubectl create ns osii-test
kubectl config set-context tools-rmi-poc-cco-dev-aks-admin --namespace=osii-test

Get Deployments in namespace default

In [ ]:
%%bash
kubectl get deployment

Create a deployment from commandline 

In [ ]:
%%bash
kubectl create deployment --image nginx:1.14.2 --port 80 nginx

Show content of deployment

In [ ]:
%%bash
kubectl get deployment nginx -o yaml

Update image version

In [ ]:
%%bash
kubectl set image deployment/nginx nginx=nginx:1.27.3

show all resources

In [ ]:
%%bash
kubectl get all

expose service nginx

In [ ]:
%%bash
kubectl expose deployment nginx --port 80 --type LoadBalancer --target-port 80 --protocol TCP

Show exposed service

In [ ]:
%%bash
kubectl get service/nginx
echo "-------------------------------"
kubectl get service/nginx -o yaml

Expose with ingress

In [ ]:
%%bash
kubectl expose deployment nginx --port 80 --target-port 80 --protocol TCP
kubectl create -f manifest/ingress-demo.yaml

Show in K9S

Using the Metrics server (Not that practical)

In [ ]:
%%bash
kubectl get --raw "/apis/metrics.k8s.io/v1beta1/pods/" | jq '[.items[] |select(.metadata.namespace |contains("osii-test"))] | .[]'

In [ ]:
%%bash
kubectl get pods.metrics.k8s.io

Installing cloudnativePG using helm 
[See Helm chart options](https://github.com/cloudnative-pg/charts/blob/main/charts/cluster/README.md)

In [ ]:
%%bash
helm repo add cnpg https://cloudnative-pg.github.io/charts
helm upgrade --install cnpg --namespace cnpg-system --create-namespace cnpg/cloudnative-pg

Create a postgresql cluster

In [ ]:
%%bash
kubectl create -f manifest/pgcluster.yaml